In [184]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np 
import pdb

In [186]:
soup = BeautifulSoup(requests.get("https://nips.cc/Conferences/2017/Schedule").content, 'html5lib')

In [190]:
poster_tables = soup.find_all('div', {'class':'maincard narrower Poster'})

In [50]:
papers = pd.DataFrame(index=range(len(poster_tables)),
                      columns=['title', 'authors', 'paper_url', 'code_url', 'poster_url'])

In [45]:
def get_url(table_no, title):
    table_section = table_index.find('a', {'title':title})
    if table_section is not None:
        return table_section['href']

In [51]:
for p in papers.index:
    title = poster_tables[p].find('div',{'class':'maincardBody'}).contents[0]
    authors = poster_tables[p].find('div',{'class':'maincardFooter'}).contents[0]
    pdf = get_url(poster_tables[p], 'Paper')
    repo = get_url(poster_tables[p], 'Code')
    poster_pdf = get_url(poster_tables[p], 'Poster')
    papers.loc[p] = pd.Series(dict(zip(papers.columns,
                                       [title, authors, pdf, repo, poster_pdf])))

In [63]:
code_papers = papers[papers['code_url'].notnull()]

In [67]:
code_papers['github'] = code_papers['code_url'].str.contains("github")

/Users/forde/miniconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [78]:
def test_url(page_name):
    with suppress(Exception):
        return requests.get(page_name).status_code == requests.codes.ok

In [ ]:
code_papers['live'] = [test_url(url) for url in code_papers['code_url']] 

In [106]:
code_papers[code_papers['live'] != True].to_csv('validate_url.csv')

In [116]:
live_gh = code_papers[np.logical_and(code_papers['github'], code_papers['live'] == True)]

In [254]:
file_types = ['Dockerfile', 'binder', 'apt.txt', 'environment.yml',
              'requirements.txt', 'postBuild', 'setup.py']
social_vals = ['languge', 'watchers', 'stargazers', 'forks']

In [255]:
def request_inspect_for_r2d(repo_url, file_types=file_types):
    repo_name = repo_url[len('https://github.com'):]
    repo_soup = BeautifulSoup(requests.get(repo_url).content, "html5lib")
    result_dict = {}
    for t in file_types:
        find_in_repo = repo_soup.find("a", {'class':'js-navigation-open', 'title':t})
        if find_in_repo is not None:
            if t == 'binder':
                check_path = '/tree/master/'
            else:
                check_path = '/blob/master/'
            result_dict[t] = find_in_repo['href'] == '{}{}{}'.format(repo_name, check_path, t)
        else: 
            result_dict[t] = False
    return result_dict

In [256]:
r2d_checks = pd.DataFrame(index=live_gh['code_url'], columns=file_types)

In [ ]:
for i in r2d_checks.index:
    r2d_checks.loc[i] = pd.Series(request_inspect_for_r2d(i))

In [208]:
r2d_checks.to_csv('gh_r2d_data.csv')

In [212]:
gh_labeled = pd.read_csv('validate_url_w_labels.csv', index_col=0)

In [242]:
r2d_labeled = pd.DataFrame(index=gh_labeled[gh_labeled['labeled_url'].str[:19] == 'https://github.com/']
                           ['labeled_url'], columns=file_types)

In [243]:
for i in r2d_labeled.index:
    if i[:19] == 'https://github.com/':
        r2d_labeled.loc[i] = pd.Series(request_inspect_for_r2d(i))

In [245]:
r2d_labeled.to_csv('labeled_r2d_data.csv')

In [246]:
full_r2d = r2d_checks.append(r2d_labeled)

In [253]:
full_r2d[full_r2d['environment.yml']]

,Dockerfile,binder,apt.txt,environment.yml,requirements.txt,postBuild
https://github.com/KarenUllrich/Tutorial_BayesianCompressionForDL,False,False,False,True,False,False
https://github.com/shaneshixiang/rllabplusplus,False,False,False,True,False,False
https://github.com/andrewgordonwilson/bayesgan,False,False,False,True,False,False
https://github.com/flatironinstitute/CaImAn,True,False,False,True,False,False
